<a href="https://colab.research.google.com/github/L0Lz-z/EVALUATING-en_core_web_md-NER-WITH-SPACY/blob/master/Model_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive' , force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

import pandas as pd
from tqdm import tqdm

In [ ]:
df = pd.read_csv('/content/drive/My Drive/Intern/datasets/ner_datasetreference.csv',encoding='latin1')


In [ ]:
df = df.fillna(method = 'ffill')


In [ ]:
class sentencs(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [[w, p, t] for w, p, t in zip(s['Word'].values.tolist(),
                                                       s['POS'].values.tolist(),
                                                       s['Tag'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None

In [ ]:
getter = sentenc(df)
sente = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sente[0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
sente[1]

'Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .'

In [ ]:
sentences = getter.sentences


In [ ]:
# pip install pandas


In [ ]:
df=pd.DataFrame(sentences,index=None)

In [ ]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,"[Thousands, NNS, O]","[of, IN, O]","[demonstrators, NNS, O]","[have, VBP, O]","[marched, VBN, O]","[through, IN, O]","[London, NNP, B-geo]","[to, TO, O]","[protest, VB, O]","[the, DT, O]","[war, NN, O]","[in, IN, O]","[Iraq, NNP, B-geo]","[and, CC, O]","[demand, VB, O]","[the, DT, O]","[withdrawal, NN, O]","[of, IN, O]","[British, JJ, B-gpe]","[troops, NNS, O]","[from, IN, O]","[that, DT, O]","[country, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"[Iranian, JJ, B-gpe]","[officials, NNS, O]","[say, VBP, O]","[they, PRP, O]","[expect, VBP, O]","[to, TO, O]","[get, VB, O]","[access, NN, O]","[to, TO, O]","[sealed, JJ, O]","[sensitive, JJ, O]","[parts, NNS, O]","[of, IN, O]","[the, DT, O]","[plant, NN, O]","[Wednesday, NNP, B-tim]","[,, ,, O]","[after, IN, O]","[an, DT, O]","[IAEA, NNP, B-org]","[surveillance, NN, O]","[system, NN, O]","[begins, VBZ, O]","[functioning, VBG, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"[Helicopter, NN, O]","[gunships, NNS, O]","[Saturday, NNP, B-tim]","[pounded, VBD, O]","[militant, JJ, O]","[hideouts, NNS, O]","[in, IN, O]","[the, DT, O]","[Orakzai, NNP, B-geo]","[tribal, JJ, O]","[region, NN, O]","[,, ,, O]","[where, WRB, O]","[many, JJ, O]","[Taliban, NNP, B-org]","[militants, NNS, O]","[are, VBP, O]","[believed, VBN, O]","[to, TO, O]","[have, VB, O]","[fled, VBN, O]","[to, TO, O]","[avoid, VB, O]","[an, DT, O]","[earlier, JJR, O]","[military, JJ, O]","[offensive, NN, O]","[in, IN, O]","[nearby, JJ, O]","[South, NNP, B-geo]","[Waziristan, NNP, I-geo]","[., ., O]",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"[They, PRP, O]","[left, VBD, O]","[after, IN, O]","[a, DT, O]","[tense, NN, O]","[hour-long, JJ, O]","[standoff, NN, O]","[with, IN, O]","[riot, NN, O]","[police, NNS, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"[U.N., NNP, B-geo]","[relief, NN, O]","[coordinator, NN, O]","[Jan, NNP, B-per]","[Egeland, NNP, I-per]","[said, VBD, O]","[Sunday, NNP, B-tim]","[,, ,, O]","[U.S., NNP, B-geo]","[,, ,, O]","[Indonesian, JJ, B-gpe]","[and, CC, O]","[Australian, JJ, B-gpe]","[military, JJ, O]","[helicopters, NNS, O]","[are, VBP, O]","[ferrying, VBG, O]","[out, RP, O]","[food, NN, O]","[and, CC, O]","[supplies, NNS, O]","[to, TO, O]","[remote, VB, O]","[areas, NNS, O]","[of, IN, O]","[western, JJ, O]","[Aceh, NNP, B-geo]","[province, NN, O]","[that, IN, O]","[ground, NN, O]","[crews, NNS, O]","[can, MD, O]","[not, RB, O]","[reach, VB, O]","[., ., O]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
type(df[0][0])

list

In [ ]:
global counter
counter=0

In [ ]:
def create(row):
    global counter
    a=sente[counter]
    counter+=1
    return a
    

In [ ]:
df['sentence']=df.apply(lambda x:create(x),axis=1)

In [ ]:
# cols = df.columns.tolist()
# cols= cols[-1:] + cols[:-1]


In [ ]:
df=df[cols]
df['sentence'][0]

'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

In [ ]:
df

,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,Thousands of demonstrators have marched throug...,"[Thousands, NNS, O]","[of, IN, O]","[demonstrators, NNS, O]","[have, VBP, O]","[marched, VBN, O]","[through, IN, O]","[London, NNP, B-geo]","[to, TO, O]","[protest, VB, O]","[the, DT, O]","[war, NN, O]","[in, IN, O]","[Iraq, NNP, B-geo]","[and, CC, O]","[demand, VB, O]","[the, DT, O]","[withdrawal, NN, O]","[of, IN, O]","[British, JJ, B-gpe]","[troops, NNS, O]","[from, IN, O]","[that, DT, O]","[country, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Iranian officials say they expect to get acces...,"[Iranian, JJ, B-gpe]","[officials, NNS, O]","[say, VBP, O]","[they, PRP, O]","[expect, VBP, O]","[to, TO, O]","[get, VB, O]","[access, NN, O]","[to, TO, O]","[sealed, JJ, O]","[sensitive, JJ, O]","[parts, NNS, O]","[of, IN, O]","[the, DT, O]","[plant, NN, O]","[Wednesday, NNP, B-tim]","[,, ,, O]","[after, IN, O]","[an, DT, O]","[IAEA, NNP, B-org]","[surveillance, NN, O]","[system, NN, O]","[begins, VBZ, O]","[functioning, VBG, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Helicopter gunships Saturday pounded militant ...,"[Helicopter, NN, O]","[gunships, NNS, O]","[Saturday, NNP, B-tim]","[pounded, VBD, O]","[militant, JJ, O]","[hideouts, NNS, O]","[in, IN, O]","[the, DT, O]","[Orakzai, NNP, B-geo]","[tribal, JJ, O]","[region, NN, O]","[,, ,, O]","[where, WRB, O]","[many, JJ, O]","[Taliban, NNP, B-org]","[militants, NNS, O]","[are, VBP, O]","[believed, VBN, O]","[to, TO, O]","[have, VB, O]","[fled, VBN, O]","[to, TO, O]","[avoid, VB, O]","[an, DT, O]","[earlier, JJR, O]","[military, JJ, O]","[offensive, NN, O]","[in, IN, O]","[nearby, JJ, O]","[South, NNP, B-geo]","[Waziristan, NNP, I-geo]","[., ., O]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,They left after a tense hour-long standoff wit...,"[They, PRP, O]","[left, VBD, O]","[after, IN, O]","[a, DT, O]","[tense, NN, O]","[hour-long, JJ, O]","[standoff, NN, O]","[with, IN, O]","[riot, NN, O]","[police, NNS, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,U.N. relief coordinator Jan Egeland said Sunda...,"[U.N., NNP, B-geo]","[relief, NN, O]","[coordinator, NN, O]","[Jan, NNP, B-per]","[Egeland, NNP, I-per]","[said, VBD, O]","[Sunday, NNP, B-tim]","[,, ,, O]","[U.S., NNP, B-geo]","[,, ,, O]","[Indonesian, JJ, B-gpe]","[and, CC, O]","[Australian, JJ, B-gpe]","[military, JJ, O]","[helicopters, NNS, O]","[are, VBP, O]","[ferrying, VBG, O]","[out, RP, O]","[food, NN, O]","[and, CC, O]","[supplies, NNS, O]","[to, TO, O]","[remote, VB, O]","[areas, NNS, O]","[of, IN, O]","[western, JJ, O]","[Aceh, NNP, B-geo]","[province, NN, O]","[that, IN, O]","[ground, NN, O]","[crews, NNS, O]","[can, MD, O]","[not, RB, O]","[reach, VB, O]","[., ., O]",None,None,None,None,...,None,None,None,None,None,None,None,None,

In [ ]:
!pip install spacy
!python -m spacy download en_core_web_md


     |████████████████████████████████| 96.4MB 2.1MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-cp36-none-any.whl size=98051305 sha256=63d65b578ba19ea5a394f0e04a9e95c6cd1e9829c1434c4d541ba88bc9b585dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-osc378kw/wheels/df/94/ad/f5cf59224cea6b5686ac4fd1ad19c8a07bc026e13c36502d81
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [ ]:
import en_core_web_md
nlp = en_core_web_md.load()


In [ ]:
text=nlp(df['sentence'][0])

In [ ]:
[ (ent.text, ent.label_) for ent in text.ents]

[('Thousands', 'CARDINAL'),
 ('London', 'GPE'),
 ('Iraq', 'GPE'),
 ('British', 'NORP')]

In [ ]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [ ]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
def evaluate(model, examples):
  scorer = Scorer()
  for input_, annot in examples:
    print(input_)
    doc_gold_text = model.make_doc(input_)
    gold = GoldParse(doc_gold_text, entities=annot['entities'])
    pred_value = model(input_)
    scorer.score(pred_value, gold)
  return scorer.scores

Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .

In [ ]:
# from os import path, mkdir
# if not path.isdir("data/"):
#     mkdir("data/")
# if not path.isdir("models/"):
#     mkdir("models/")

In [ ]:
# !curl https://groups.csail.mit.edu/sls/downloads/movie/engtrain.bio -o data/train.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  989k  100  989k    0     0   277k      0  0:00:03  0:00:03 --:--:--  277k


In [ ]:
# def load_data_spacy(file_path):
#     ''' Converts data from:
#     label \t word \n label \t word \n \n label \t word
#     to: sentence, {entities : [(start, end, label), (stard, end, label)]}
#     '''
#     file = open(file_path, 'r')
#     training_data, entities, sentence, unique_labels = [], [], [], []
#     current_annotation = None
#     end = 0 # initialize counter to keep track of start and end characters
#     for line in file:
#         line = line.strip("\n").split("\t")
#         # lines with len > 1 are words
#         if len(line) > 1:
#             label = line[0][2:]     # the .txt is formatted: label \t word, label[0:2] = label_type
#             label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
#             word = line[1]
#             sentence.append(word)
#             end += (len(word) + 1)  # length of the word + trailing space
           
#             if label_type != 'I' and current_annotation:  # if at the end of an annotation
#                 entities.append((start, end - 2 - len(word), current_annotation))  # append the annotation
#                 current_annotation = None                 # reset the annotation
#             if label_type == 'B':                         # if beginning new annotation
#                 start = end - len(word) - 1  # start annotation at beginning of word
#                 current_annotation = label   # append the word to the current annotation
#             if label_type == 'I':            # if the annotation is multi-word
#                 current_annotation = label   # append the word
           
#             if label != 'O' and label not in unique_labels:
#                 unique_labels.append(label)
 
#         # lines with len == 1 are breaks between sentences
#         if len(line) == 1:
#             if current_annotation:
#                 entities.append((start, end - 1, current_annotation))
#             sentence = " ".join(sentence)
#             training_data.append([sentence, {'entities' : entities}])
#             # reset the counters and temporary lists
#             end = 0            
#             entities, sentence = [], []
#             current_annotation = None
#     file.close()
#     return training_data, unique_labels            
           
# TRAIN_DATA, LABELS = load_data_spacy("data/train.txt")

In [ ]:
# file=open("data/train.txt",'r')

In [ ]:
import json
with open('/content/drive/My Drive/Intern/NER_data.json','r') as outfile:
  TRAIN_DATA = json.loads(outfile.read())


In [ ]:
len(TRAIN_DATA)

18565

In [ ]:
# from __future__ import unicode_literals, print_function

# import random
# import warnings
# from pathlib import Path
# import spacy
# from spacy import displacy
# from time import time
# from spacy.util import minibatch, compounding
# spacy.prefer_gpu()

# def main(model=None,TRAIN_DATA=TRAIN_DATA, new_model_name="sams_en_ner", output_dir="sams_en_ner", n_iter=50,labels=['PERSON','NPERSON']):
#     """Set up the pipeline and entity recognizer, and train the new entity."""
#     random.seed(0)
#     if model is not None:
#         nlp = spacy.load(model)  # load existing spaCy model
#         print("Loaded model '%s'" % model)
#     else:
#         nlp = spacy.blank("en")  # create blank Language class
#         print("Created blank 'en' model")
#     # Add entity recognizer to model if it's not in the pipeline
#     # nlp.create_pipe works for built-ins that are registered with spaCy
#     if "ner" not in nlp.pipe_names:
#         ner = nlp.create_pipe("ner")
#         nlp.add_pipe(ner)
#         optimizer = nlp.begin_training()
#         for LABEL in labels:
#           ner.add_label(LABEL)  
#     # otherwise, get it, so we can add labels to it
#     else:
#       ner = nlp.get_pipe("ner")
#       optimizer = nlp.resume_training()
      
#     # if model is None:
#     #     optimizer = nlp.begin_training()
#     # else:
#     #     optimizer = nlp.resume_training()
#     move_names = list(ner.move_names)
#     # get names of other pipes to disable them during training
#     pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
#     other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
#     # only train NER
#     with nlp.disable_pipes(*other_pipes) and warnings.catch_warnings():
#         # show warnings for misaligned entity spans once
#         warnings.filterwarnings("once", category=UserWarning, module='spacy')

#         sizes = compounding(1.0, 4.0, 1.001)
#         # batch up the examples using spaCy's minibatch
#         for itn in range(n_iter):
#             start = time()
#             random.shuffle(TRAIN_DATA)
#             TRAIN_DATA = TRAIN_DATA[:5000]
#             batches = minibatch(TRAIN_DATA, size=sizes)
#             losses = {}
#             for batch in batches:
#                 texts, annotations = zip(*batch)
#                 nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
#             print(f"Iter : {itn:{3}} | Loss : {losses['ner']:{20}} | Time : {time()-start:{20}}")

#     # test the trained model
#     test_text = "Alan John is living in Pune since 2015."
#     doc = nlp(test_text)
#     colors = {"PERSON": "linear-gradient(90deg, #aa9cfc, #fc9ce7)" , "NPERSON": "linear-gradient(90deg, #fabc46, #48e061)"}
#     options = {"ents": ["PERSON","NPERSON"], "colors": colors}
#     displacy.render(doc, style="ent",jupyter=True, options=options)
#     print(nlp.pipe_names)
#     # save model to output directory
#     if output_dir is not None:
#         output_dir = Path(output_dir)
#         if not output_dir.exists():
#             output_dir.mkdir()
#         nlp.meta["name"] = new_model_name  # rename model
#         nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)

#         # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)
#         # Check the classes have loaded back consistently
#         assert nlp2.get_pipe("ner").move_names == move_names
#         doc2 = nlp2(test_text)
#         for ent in doc2.ents:
#             print(ent.label_, ent.text)

In [ ]:
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
def evaluate(model, examples):
  scorer = Scorer()
  for input_, annot in examples:
    #print(input_)
    doc_gold_text = model.make_doc(input_)
    gold = GoldParse(doc_gold_text, entities=annot['entities'])
    pred_value = model(input_)
    scorer.score(pred_value, gold)
  return scorer.scores

In [ ]:
evaluate(nlp,TRAIN_DATA)

{'ents_f': 11.46766214899134,
 'ents_p': 9.666118981926404,
 'ents_per_type': {'CARDINAL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'DATE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'EVENT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'FAC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'GPE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LANGUAGE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LAW': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LOC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'MONEY': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'NORP': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'NPERSON': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'ORDINAL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'ORG': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PERCENT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PERSON': {'f': 46.422988163478,
   'p': 36.37848559094621,
   'r': 64.12998765939942},
  'PRODUCT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'QUANTITY': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'TIME': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'WORK_OF_ART': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'ents_r': 14.094566494891964,
 'las': 0.0,
 'la

In [ ]:
df

,0,1
0,Pak PM issues orders to keep Maryam Nawaz with...,"{'entities': [[0, 6, 'PERSON'], [29, 41, 'PERS..."
1,Maryam was admitted to the Services Hospital i...,"{'entities': [[0, 6, 'PERSON'], [23, 44, 'NPER..."
2,The 45-year-old leader was admitted to VVIP ll...,"{'entities': [[39, 46, 'NPERSON']]}"
3,She underwent some tests after which she was s...,{'entities': []}
4,"Expression patterns of seven key genes, includ...","{'entities': [[23, 28, 'NPERSON'], [86, 89, 'N..."
...,...,...
18560,"On September 3, an IED struck a civilian bus i...","{'entities': [[3, 14, 'NPERSON'], [48, 53, 'NP..."
18561,JNIM later issued an apology stating it regret...,"{'entities': [[0, 4, 'NPERSON'], [131, 137, 'N..."
18562,"On September 30, near the Burkina Faso border,...","{'entities': [[3, 15, 'NPERSON'], [26, 38, 'NP..."
18563,JNIM subsequently took responsibility for the ...,"{'entities': [[0, 4, 'NPERSON'], [78, 80, 'NPE..."


In [ ]:
df

,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103
0,Thousands of demonstrators have marched throug...,"[Thousands, NNS, O]","[of, IN, O]","[demonstrators, NNS, O]","[have, VBP, O]","[marched, VBN, O]","[through, IN, O]","[London, NNP, B-geo]","[to, TO, O]","[protest, VB, O]","[the, DT, O]","[war, NN, O]","[in, IN, O]","[Iraq, NNP, B-geo]","[and, CC, O]","[demand, VB, O]","[the, DT, O]","[withdrawal, NN, O]","[of, IN, O]","[British, JJ, B-gpe]","[troops, NNS, O]","[from, IN, O]","[that, DT, O]","[country, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,Iranian officials say they expect to get acces...,"[Iranian, JJ, B-gpe]","[officials, NNS, O]","[say, VBP, O]","[they, PRP, O]","[expect, VBP, O]","[to, TO, O]","[get, VB, O]","[access, NN, O]","[to, TO, O]","[sealed, JJ, O]","[sensitive, JJ, O]","[parts, NNS, O]","[of, IN, O]","[the, DT, O]","[plant, NN, O]","[Wednesday, NNP, B-tim]","[,, ,, O]","[after, IN, O]","[an, DT, O]","[IAEA, NNP, B-org]","[surveillance, NN, O]","[system, NN, O]","[begins, VBZ, O]","[functioning, VBG, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,Helicopter gunships Saturday pounded militant ...,"[Helicopter, NN, O]","[gunships, NNS, O]","[Saturday, NNP, B-tim]","[pounded, VBD, O]","[militant, JJ, O]","[hideouts, NNS, O]","[in, IN, O]","[the, DT, O]","[Orakzai, NNP, B-geo]","[tribal, JJ, O]","[region, NN, O]","[,, ,, O]","[where, WRB, O]","[many, JJ, O]","[Taliban, NNP, B-org]","[militants, NNS, O]","[are, VBP, O]","[believed, VBN, O]","[to, TO, O]","[have, VB, O]","[fled, VBN, O]","[to, TO, O]","[avoid, VB, O]","[an, DT, O]","[earlier, JJR, O]","[military, JJ, O]","[offensive, NN, O]","[in, IN, O]","[nearby, JJ, O]","[South, NNP, B-geo]","[Waziristan, NNP, I-geo]","[., ., O]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,They left after a tense hour-long standoff wit...,"[They, PRP, O]","[left, VBD, O]","[after, IN, O]","[a, DT, O]","[tense, NN, O]","[hour-long, JJ, O]","[standoff, NN, O]","[with, IN, O]","[riot, NN, O]","[police, NNS, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,U.N. relief coordinator Jan Egeland said Sunda...,"[U.N., NNP, B-geo]","[relief, NN, O]","[coordinator, NN, O]","[Jan, NNP, B-per]","[Egeland, NNP, I-per]","[said, VBD, O]","[Sunday, NNP, B-tim]","[,, ,, O]","[U.S., NNP, B-geo]","[,, ,, O]","[Indonesian, JJ, B-gpe]","[and, CC, O]","[Australian, JJ, B-gpe]","[military, JJ, O]","[helicopters, NNS, O]","[are, VBP, O]","[ferrying, VBG, O]","[out, RP, O]","[food, NN, O]","[and, CC, O]","[supplies, NNS, O]","[to, TO, O]","[remote, VB, O]","[areas, NNS, O]","[of, IN, O]","[western, JJ, O]","[Aceh, NNP, B-geo]","[province, NN, O]","[that, IN, O]","[ground, NN, O]","[crews, NNS, O]","[can, MD, O]","[not, RB, O]","[reach, VB, O]","[., ., O]",None,None,None,None,...,None,None,None,None,None,None,None,None,

In [ ]:
cols=df.columns.to_list()
cols=cols[1:-1]
cols

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103]

In [ ]:
NEW_DATA=[]
def create_spacy(row):
    text=row['sentence']
    result = {'entities' : []}
    count=0
    for i in cols:
        if row[i]!=None:
            if row[i][2]!='O':
                start=count
                end=start+len(row[i][0])
                
                # label = 'PERSON' if (row[i][2] == 'B-per'or row[i][2]=='I-per') else 'NPERSON'
                if (row[i][2] == 'B-per'):
                    if ( row[i+1]!=None and row[i+1][2]=='I-per' ):
                            if (row[i+2]!=None and row[i+2][2]=='I-per'):
                                start=count
                                end=start+len(row[i][0])+len(row[i+1][0])+len(row[i+2][0])+2
                                label='PERSON'
                            else:
                                start=count
                                end=start+len(row[i][0])+len(row[i+1][0])+1
                                label='PERSON'
                    else:
                        start=count
                        end=start+len(row[i][0])
                        label='PERSON'
                else:
                    label='NPERSON'
                count=end+1
                result['entities'].append((start,end,label))
            else:
                count=count+len(row[i][0])+1
        else:
            pass
    return (text,result)

In [ ]:
df['new']=df.apply(lambda x:create_spacy(x),axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df

,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,new
0,Thousands of demonstrators have marched throug...,"[Thousands, NNS, O]","[of, IN, O]","[demonstrators, NNS, O]","[have, VBP, O]","[marched, VBN, O]","[through, IN, O]","[London, NNP, B-geo]","[to, TO, O]","[protest, VB, O]","[the, DT, O]","[war, NN, O]","[in, IN, O]","[Iraq, NNP, B-geo]","[and, CC, O]","[demand, VB, O]","[the, DT, O]","[withdrawal, NN, O]","[of, IN, O]","[British, JJ, B-gpe]","[troops, NNS, O]","[from, IN, O]","[that, DT, O]","[country, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(Thousands of demonstrators have marched throu...
1,Iranian officials say they expect to get acces...,"[Iranian, JJ, B-gpe]","[officials, NNS, O]","[say, VBP, O]","[they, PRP, O]","[expect, VBP, O]","[to, TO, O]","[get, VB, O]","[access, NN, O]","[to, TO, O]","[sealed, JJ, O]","[sensitive, JJ, O]","[parts, NNS, O]","[of, IN, O]","[the, DT, O]","[plant, NN, O]","[Wednesday, NNP, B-tim]","[,, ,, O]","[after, IN, O]","[an, DT, O]","[IAEA, NNP, B-org]","[surveillance, NN, O]","[system, NN, O]","[begins, VBZ, O]","[functioning, VBG, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(Iranian officials say they expect to get acce...
2,Helicopter gunships Saturday pounded militant ...,"[Helicopter, NN, O]","[gunships, NNS, O]","[Saturday, NNP, B-tim]","[pounded, VBD, O]","[militant, JJ, O]","[hideouts, NNS, O]","[in, IN, O]","[the, DT, O]","[Orakzai, NNP, B-geo]","[tribal, JJ, O]","[region, NN, O]","[,, ,, O]","[where, WRB, O]","[many, JJ, O]","[Taliban, NNP, B-org]","[militants, NNS, O]","[are, VBP, O]","[believed, VBN, O]","[to, TO, O]","[have, VB, O]","[fled, VBN, O]","[to, TO, O]","[avoid, VB, O]","[an, DT, O]","[earlier, JJR, O]","[military, JJ, O]","[offensive, NN, O]","[in, IN, O]","[nearby, JJ, O]","[South, NNP, B-geo]","[Waziristan, NNP, I-geo]","[., ., O]",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(Helicopter gunships Saturday pounded militant...
3,They left after a tense hour-long standoff wit...,"[They, PRP, O]","[left, VBD, O]","[after, IN, O]","[a, DT, O]","[tense, NN, O]","[hour-long, JJ, O]","[standoff, NN, O]","[with, IN, O]","[riot, NN, O]","[police, NNS, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(They left after a tense hour-long standoff wi...
4,U.N. relief coordinator Jan Egeland said Sunda...,"[U.N., NNP, B-geo]","[relief, NN, O]","[coordinator, NN, O]","[Jan, NNP, B-per]","[Egeland, NNP, I-per]","[said, VBD, O]","[Sunday, NNP, B-tim]","[,, ,, O]","[U.S., NNP, B-geo]","[,, ,, O]","[Indonesian, JJ, B-gpe]","[and, CC, O]","[Australian, JJ, B-gpe]","[military, JJ, O]","[helicopters, NNS, O]","[are, VBP, O]","[ferrying, VBG, O]","[out, RP, O]","[food, NN, O]","[and, CC, O]","[supplies, NNS, O]","[to, TO, O]","[remote, VB, O]","[areas, NNS, O]","[of, IN, O]","[western, JJ, O]","[Aceh, NNP, B-geo]","[province, NN

In [ ]:
df.iloc[20:49,]

,sentence,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,new
20,Doctors say the stroke has not caused any perm...,"[Doctors, NNS, O]","[say, VBP, O]","[the, DT, O]","[stroke, NN, O]","[has, VBZ, O]","[not, RB, O]","[caused, VBN, O]","[any, DT, O]","[permanent, JJ, O]","[damage, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(Doctors say the stroke has not caused any per...
21,The designers of the first private manned rock...,"[The, DT, O]","[designers, NNS, O]","[of, IN, O]","[the, DT, O]","[first, JJ, O]","[private, JJ, O]","[manned, VBN, O]","[rocket, NN, O]","[to, TO, O]","[burst, VB, O]","[into, IN, O]","[space, NN, O]","[have, VBP, O]","[received, VBN, O]","[a, DT, O]","[$, $, O]","[10, CD, O]","[million, CD, O]","[prize, NN, O]","[created, VBN, O]","[to, TO, O]","[promote, VB, O]","[space, NN, O]","[tourism, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(The designers of the first private manned roc...
22,SpaceShipOne designer Burt Rutan accepted the ...,"[SpaceShipOne, NNP, B-art]","[designer, NN, O]","[Burt, NNP, B-per]","[Rutan, NNP, I-per]","[accepted, VBD, O]","[the, DT, O]","[Ansari, NNP, B-art]","[X, NNP, I-art]","[Prize, NNP, I-art]","[money, NN, O]","[and, CC, O]","[a, DT, O]","[trophy, NN, O]","[on, IN, O]","[behalf, NN, O]","[of, IN, O]","[his, PRP$, O]","[team, NN, O]","[Saturday, NNP, B-tim]","[during, IN, O]","[an, DT, O]","[awards, NNS, O]","[ceremony, NN, O]","[in, IN, O]","[the, DT, O]","[U.S., NNP, B-geo]","[state, NN, O]","[of, IN, O]","[Missouri, NNP, B-geo]","[., ., O]",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,(SpaceShipOne designer Burt Rutan accepted the...
23,"To win the money , SpaceShipOne had to blast o...","[To, TO, O]","[win, VB, O]","[the, DT, O]","[money, NN, O]","[,, ,, O]","[SpaceShipOne, NNP, B-art]","[had, VBD, O]","[to, TO, O]","[blast, VB, O]","[off, RP, O]","[into, IN, O]","[space, NN, O]","[twice, RB, O]","[in, IN, O]","[a, DT, O]","[two-week, JJ, B-tim]","[period, NN, I-tim]","[and, CC, O]","[fly, VB, O]","[at, IN, O]","[least, JJS, O]","[100, CD, O]","[kilometers, NNS, O]","[above, IN, O]","[Earth, NNP, B-geo]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,"(To win the money , SpaceShipOne had to blast ..."
24,The spacecraft made its flights in late Septem...,"[The, DT, O]","[spacecraft, NN, O]","[made, VBD, O]","[its, PRP$, O]","[flights, NNS, O]","[in, IN, O]","[late, JJ, O]","[September, NNP, B-tim]","[and, CC, O]","[early, JJ, O]","[October, NNP, B-tim]","[,, ,, O]","[lifting, VBG, O]","[off, RP, O]","[from, IN, O]","[California, NNP, B-geo]","['s, POS, O]","[Mojave, NNP, B-geo]","[desert, NN, O]","[., ., O]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [ ]:
for i in df['new']:
    NEW_DATA.append(i)

In [ ]:
NEW_DATA

[('Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
  {'entities': [(48, 54, 'NPERSON'),
    (77, 81, 'NPERSON'),
    (111, 118, 'NPERSON')]}),
 ('Iranian officials say they expect to get access to sealed sensitive parts of the plant Wednesday , after an IAEA surveillance system begins functioning .',
  {'entities': [(0, 7, 'NPERSON'),
    (87, 96, 'NPERSON'),
    (108, 112, 'NPERSON')]}),
 ('Helicopter gunships Saturday pounded militant hideouts in the Orakzai tribal region , where many Taliban militants are believed to have fled to avoid an earlier military offensive in nearby South Waziristan .',
  {'entities': [(20, 28, 'NPERSON'),
    (62, 69, 'NPERSON'),
    (97, 104, 'NPERSON'),
    (190, 195, 'NPERSON'),
    (196, 206, 'NPERSON')]}),
 ('They left after a tense hour-long standoff with riot police .',
  {'entities': []}),
 ('U.N. relief coordinator Jan Egeland said Sunday , U.S. , I

**PRE TRAINED SPACY MODEL...en_core_web_md**

In [ ]:
evaluate(nlp,NEW_DATA)

{'ents_f': 1.277061849165367,
 'ents_p': 1.2874531835205993,
 'ents_per_type': {'CARDINAL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'DATE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'EVENT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'FAC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'GPE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LANGUAGE': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LAW': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'LOC': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'MONEY': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'NORP': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'NPERSON': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'ORDINAL': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'ORG': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PERCENT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'PERSON': {'f': 33.37290594908323,
   'p': 30.674102812803106,
   'r': 36.592421174428694},
  'PRODUCT': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'QUANTITY': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'TIME': {'f': 0.0, 'p': 0.0, 'r': 0.0},
  'WORK_OF_ART': {'f': 0.0, 'p': 0.0, 'r': 0.0}},
 'ents_r': 1.2668369135246107,
 'las': 0.0,

In [ ]:
text=nlp('The new head of the Palestinian Fatah movement has threatened to expel popular leader Marwan Barghouti unless he withdraws from the Palestinian presidential race .')

In [ ]:
[print(i.ents,i.label_)for i in text.ents]

[Palestinian] NORP
[Fatah] ORG
[Marwan Barghouti] PERSON
[Palestinian] NORP


[None, None, None, None]

In [ ]:
s='The new head of the Palestinian Fatah movement has threatened to expel popular leader Marwan Barghouti unless he withdraws from the Palestinian presidential race .'

In [288]:
s[86:102]

'Marwan Barghouti'

#  **sams_en_ner_sm**

In [ ]:
output_dir="/content/drive/My Drive/Intern/sams_en_ner_sm"
output_dir = Path(output_dir)
nlp = spacy.load(output_dir)

In [ ]:
evaluate(nlp, TRAIN_DATA)['ents_per_type']['PERSON']

{'f': 86.44777230025113, 'p': 78.90983188996434, 'r': 95.5779514603044}